<a href="https://colab.research.google.com/github/Recommendation-System-Project/Recommendation-system/blob/main/DR_Final_Expt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib import style
import time

In [ ]:
# def get_data(sample_num=2000):
#     observed_r = np.random.uniform(low=0, high=5, size=(sample_num,)) # observed_r
#     predicted_r = np.random.uniform(low=0, high=5, size=(sample_num,)) # predicted rating
# #     imputed_e = np.power((predicted_r - 0.5), 2)*0.3 ## e_hat_ui = 0.3*(r_ui - 0.5)^2
    
#     observed_e = predicted_r - observed_r # e_ui
#     propensities = np.random.uniform(low=0, high=1, size=(sample_num,)) # propensities for observed X_ui
   
#     return observed_r, predicted_r, observed_e, propensities

In [4]:
def get_data():
    observed_r = pd.read_csv("preproceed_data/rating_pairs.csv").drop("user_id", axis = 1) # observed ratings
    observed_r.columns = range(1,len(observed_r.columns)+1)
    
    predicted_r = pd.DataFrame(np.random.randint(1,5,size=observed_r.shape)) # predicted rating
    predicted_r.columns = range(1,len(predicted_r.columns)+1)
    
    observed_e = predicted_r - observed_r # e_ui
    propensities = pd.read_csv("preproceed_data/propensities.csv", header = None) # propensities for all user item pairs
    propensities.columns = range(1,len(propensities.columns)+1)
    
    O_mask = pd.read_csv("preproceed_data/O_mask.csv").drop("user_id", axis = 1)
    O_mask.columns = range(1,len(O_mask.columns)+1)

    item_features = np.array(pd.read_csv("preproceed_data/item_features.csv", header = None))
    user_features = np.array(pd.read_csv("preproceed_data/user_features.csv", header = None))
   
    return observed_r, predicted_r, observed_e, propensities, O_mask, item_features, user_features

In [ ]:
def SGD(predicted_r,
        observed_e,
        propensities,
        step_size=2,
        max_iter_count=2000):

    m = predicted_r.shape[0]
    var = 1
    loss = 1
    iter_count = 0
    iter_list = []
    loss_list = []
    theta1 = []
    theta2 = []
    theta = [2.2, 0.58]

    
    while loss > 0.01 and iter_count < max_iter_count:
        loss = 0
        theta1.append(theta[0])
        theta2.append(theta[1])
        rand1 = np.random.randint(0, m, 1)

        gradient_w = 2 * predicted_r[rand1] * (
            2 * theta[1] - predicted_r[rand1]) * (
                (2 * predicted_r[rand1] * theta[1] - predicted_r[rand1]**2) *
                theta[0] + observed_e[rand1] - theta[1]**2) / propensities[rand1]

        gradient_y = 4 * (theta[1] - theta[0] * predicted_r[rand1]) * (
            theta[1]**2 - 2 * predicted_r[rand1] * theta[0] * theta[1] -
            observed_e[rand1] + theta[0] * predicted_r[rand1]**2) / propensities[rand1]
        
        gradient = [gradient_w, gradient_y]
        
        for i in range(len(theta)):
            theta[i] = theta[i] - step_size * gradient[i]
        h = np.power((predicted_r - theta[1]), 2) * theta[0]
        
        for i in range(m):
            every_loss = np.power((h[i] - observed_e[i]), 2) / propensities[i]
            loss = loss + every_loss

        print("iter_count: ", iter_count, "the loss: ",loss)
        print("theta:", theta, "gradient: ", gradient)

        iter_list.append(iter_count)
        loss_list.append(loss)

        iter_count += 1

    plt.plot(iter_list, loss_list)
    plt.xlabel("iter")
    plt.ylabel("loss")
    plt.show()
    return theta1, theta2, theta, loss_list

In [25]:
if __name__ == '__main__':
    observed_r, predicted_r, observed_e, propensities, O_mask, item_features, user_features = get_data()
    
    # condense data into 1d array with only observed O(u,i)
    observed_r_1d = np.array(observed_r*O_mask).flatten()
    observed_r_1d = observed_r_1d[~np.isnan(observed_r_1d)]

    predicted_r_1d = np.array(predicted_r*O_mask).flatten()
    predicted_r_1d = predicted_r_1d[predicted_r_1d != 0]
    
    observed_e_1d = np.array(observed_e).flatten()
    observed_e_1d = observed_e_1d[~np.isnan(observed_e_1d)]
    
    propensities_1d = np.array(propensities*O_mask).flatten()
    propensities_1d = propensities_1d[propensities_1d != 0]

    # creating features for all user-item pairs
    observation_matrix = []
    for u in range(user_features.shape[0]):
      for i in range(item_features.shape[0]):
        user_f = user_features[u]
        item_f = item_features[i]
        observed = np.concatenate((user_f, item_f))
        observation_matrix.append(observed)
    observation_matrix = np.array(observation_matrix)

    # select features for only observed user-item pairs
    O_mask_1d = np.array(O_mask).flatten()
    observed_features = []
    for i in range(len(O_mask_1d)):
      if O_mask_1d[i] == 1:
        observed_features.append(observation_matrix[i])
    observed_features = np.array(observed_features)

    
    # start = time.time()
    # theta1,theta2,theta,loss_list = SGD(predicted_r_1d, observed_e_1d, propensities_1d, step_size = 0.0001)
    # end = time.time()
    
    # print("The time of execution of above program is :", end-start)
    # print("theta for imputation error: ", theta)

In [6]:
from itertools import count # 迭代器
from collections import defaultdict # 使用dict时，如果引用的Key不存在，就会抛出KeyError。如果希望key不存在时，返回一个默认值，就可以用defaultdict
from scipy.sparse import csr # csr_matrix，全名为Compressed Sparse Row，是按行对矩阵进行压缩的。CSR需要三类数据：数值，列号，以及行偏移量。CSR是一种编码的方式，其中，数值与列号的含义，与coo里是一致的。行偏移表示某一行的第一个元素在values里面的起始偏移位置。 
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tqdm import tqdm_notebook as tqdm # 可以显示循环的进度条的库

Instructions for updating:
non-resource variables are not supported in the long term


In [51]:
x_train = observed_features
y_train = observed_r_1d

In [52]:
n,p = x_train.shape

k = 10

x = tf.placeholder('float',[None,p])

y = tf.placeholder('float',[None,1])

w0 = tf.Variable(tf.zeros([1]))
w = tf.Variable(tf.zeros([p]))

v = tf.Variable(tf.random_normal([k,p],mean=0,stddev=0.01))

#y_hat = tf.Variable(tf.zeros([n,1]))

linear_terms = tf.add(w0,tf.reduce_sum(tf.multiply(w,x),1,keep_dims=True)) # n * 1
pair_interactions = 0.5 * tf.reduce_sum(
    tf.subtract(
        tf.pow(
            tf.matmul(x,tf.transpose(v)),2),
        tf.matmul(tf.pow(x,2),tf.transpose(tf.pow(v,2)))
    ),axis = 1 , keep_dims=True)

y_hat = tf.add(linear_terms,pair_interactions)

In [55]:
lambda_w = tf.constant(0.001,name='lambda_w')
lambda_v = tf.constant(0.001,name='lambda_v')

l2_norm = tf.reduce_sum(
    tf.add(
        tf.multiply(lambda_w,tf.pow(w,2)),
        tf.multiply(lambda_v,tf.pow(v,2))
    )
)

error = tf.reduce_mean(tf.square(y-y_hat))
loss = tf.add(error,l2_norm)

train_op = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

In [77]:
# Launch the graph
init = tf.global_variables_initializer()
with tf.Session() as sess:
  sess.run(init)

  for epoch in tqdm(range(epochs), unit='epoch'):
    print(sess.run(train_op, feed_dict={x: x_train.reshape(-1, p), y: y_train.reshape(-1, 1)}))
    print(sess.run(loss, feed_dict={x: x_train.reshape(-1, p), y: y_train.reshape(-1, 1)}))
    

  errors = sess.run(error,feed_dict={x: x_train.reshape(-1, p), y: y_train.reshape(-1, 1)})
  print(errors)
  RMSE = np.sqrt(np.array(errors).mean())
  print(RMSE)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/10 [00:00<?, ?epoch/s]

None
4.151494e+20
None
inf
None
nan
None
nan
None
nan
None
nan
None
nan
None
nan
None
nan
None
nan
nan
nan


In [75]:
y_hat

<tf.Tensor 'Add_9:0' shape=(?, 1) dtype=float32>